In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyfigtree import figtree
from sklearn import preprocessing
from scipy.optimize import root
from scipy.integrate import odeint
epsilon = 1e-5

In [2]:
def direct_evaluation(S,T,weights,delta):
    checkPot = []
    for target in T:
        # for each source, add to potential at current target
        diff = target-S
        norm = np.linalg.norm(diff, axis=1)
        up = (norm / delta)**2
        potential = np.exp(-up)*weights
        
        cum_potential = np.sum(potential)
        checkPot.append(cum_potential)
    return np.array(checkPot)

# Form Evolution Equations
## Direct Functions

In [3]:
def direct_position(X,alpha,delta):
    X_x = direct_evaluation(X,X,alpha[:,0], delta)
    X_y = direct_evaluation(X,X,alpha[:,1], delta)
    X_z = direct_evaluation(X,X,alpha[:,2], delta) 
    return np.dstack([X_x,X_y, X_z])[0]

In [4]:
def direct_momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = direct_evaluation(X,X,first_weights[:,0],delta)
    first_sum_y = direct_evaluation(X,X,first_weights[:,1],delta)
    first_sum_z = direct_evaluation(X,X,first_weights[:,2],delta)
    
    
    second_sum_x = direct_evaluation(X,X,second_weights[:,0],delta)
    second_sum_y = direct_evaluation(X,X,second_weights[:,1],delta)
    second_sum_z = direct_evaluation(X,X,second_weights[:,2],delta)
    
    
    third_sum_x = direct_evaluation(X,X,third_weights[:,0],delta)
    third_sum_y = direct_evaluation(X,X,third_weights[:,1],delta)
    third_sum_z = direct_evaluation(X,X,third_weights[:,2],delta)
    
    fourth_sum = direct_evaluation(X,X,ax,delta)
    fifth_sum = direct_evaluation(X,X,ay,delta)
    sixth_sum = direct_evaluation(X,X,az,delta)
    
    second_term = ax * fourth_sum + ay * fifth_sum + az * sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
    
    
    mult = -2./(delta ** 2)
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

## Figtree Functions

In [5]:
def position(X,alpha,delta):
    X_x = figtree(X,X,alpha[:,0], delta,epsilon=epsilon, eval="auto")
    X_y = figtree(X,X,alpha[:,1], delta,epsilon=epsilon, eval="auto")
    X_z = figtree(X,X,alpha[:,2], delta,epsilon=epsilon, eval="auto") 
    return np.dstack([X_x,X_y, X_z])[0]

In [6]:
def momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = figtree(X,X,first_weights[:,0],delta, epsilon=epsilon, eval="auto")
    first_sum_y = figtree(X,X,first_weights[:,1],delta, epsilon=epsilon, eval="auto")
    first_sum_z = figtree(X,X,first_weights[:,2],delta, epsilon=epsilon, eval="auto")
    
    
    second_sum_x = figtree(X,X,second_weights[:,0],delta, epsilon=epsilon, eval="auto")
    second_sum_y = figtree(X,X,second_weights[:,1],delta, epsilon=epsilon, eval="auto")
    second_sum_z = figtree(X,X,second_weights[:,2],delta, epsilon=epsilon, eval="auto")
    
    third_sum_x = figtree(X,X,third_weights[:,0],delta, epsilon=epsilon, eval="auto")
    third_sum_y = figtree(X,X,third_weights[:,1],delta, epsilon=epsilon, eval="auto")
    third_sum_z = figtree(X,X,third_weights[:,2],delta, epsilon=epsilon, eval="auto")
    
    fourth_sum = figtree(X,X,ax,delta, epsilon=epsilon, eval="auto")
    fifth_sum = figtree(X,X,ay,delta, epsilon=epsilon, eval="auto")
    sixth_sum = figtree(X,X,az,delta, epsilon=epsilon, eval="auto")
    
    second_term = ax * fourth_sum + ay * fifth_sum + az * sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
 
    mult = -2./(delta ** 2)
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

# Linearizing Data

In [7]:
def f(vector, delta):
    positions, momentums = np.split(vector,2)
    positions = np.reshape(positions,(3,len(positions)/3)).T
    momentums = np.reshape(momentums,(3,len(momentums)/3)).T
    new_positions = position(positions, momentums,delta)
    new_momentums = momentum(positions, momentums,delta)
    n,m = new_positions.shape
    new_positions = np.reshape(new_positions.T, (n*m))
    new_momentums = np.reshape(new_momentums.T, (n*m))
    return np.hstack([new_positions,new_momentums])

In [8]:
def f_direct(vector, delta):
    positions, momentums = np.split(vector,2)
    positions = np.reshape(positions,(3,len(positions)/3)).T
    momentums = np.reshape(momentums,(3,len(momentums)/3)).T
    new_positions = direct_position(positions, momentums,delta)
    new_momentums = direct_momentum(positions, momentums,delta)
    n,m = new_positions.shape
    new_positions = np.reshape(new_positions.T, (n*m))
    new_momentums = np.reshape(new_momentums.T, (n*m))
    return np.hstack([new_positions,new_momentums])

# Shooting Method

In [9]:
def shoot(alpha, x_a, x_b, sigma,steps):
    #Making lambda function to integrate
    g = lambda y,t: f_direct(y,sigma)
    
    x_i = x_a
    a_i = alpha
    
    #Linealizing vectors
    n,m = x_i.shape
    x_i = np.reshape(x_i.T, (n*m))
    #a_i = np.reshape(a_i.T, (n*m))
    
    vector = np.hstack([x_i,a_i])
    #Integrate over time
    t = np.linspace(0,1,steps)
    matrix = odeint(g,vector,t)
    
    vector = matrix[-1]
    x_b = np.reshape(x_b.T, (n*m))
    x_i, momentums = np.split(vector,2)
    print np.linalg.norm(x_i - x_b)/np.linalg.norm(x_b)
    return x_i - x_b

In [10]:
def shootingMethod(x_a, x_b, alpha0, steps, sigma):
    alpha = root(shoot, alpha0, (x_a,x_b,sigma,steps))
    return alpha

# Testing

In [20]:
sigma = 6.5 * (2**0.5)
X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X0 = np.dstack((X,Y,Z))[0]

XF = np.loadtxt('data/vent_x.txt')
YF = np.loadtxt('data/vent_y.txt')
ZF = np.loadtxt('data/vent_z.txt')

X1 = np.dstack((XF,YF,ZF))[0]

A = np.loadtxt('data/alpha0_x.txt')
B = np.loadtxt('data/alpha0_y.txt')
C = np.loadtxt('data/alpha0_z.txt')

alpha_sol = np.dstack((A,B,C))[0]

In [12]:
%time postition1 = direct_position(X0,alpha_sol,sigma)

CPU times: user 3.58 s, sys: 4 ms, total: 3.58 s
Wall time: 3.58 s


In [13]:
%time postition2 = position(X0,alpha_sol,sigma)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 12.6 ms


In [14]:
%time momentum1 = direct_momentum(X0,alpha_sol,sigma)

CPU times: user 13 s, sys: 8 ms, total: 13 s
Wall time: 13 s


In [15]:
%time momentum2 = momentum(X0,alpha_sol,sigma)

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 68.5 ms


In [16]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X0[:,0],X0[:,1],X0[:,2])
plt.show()

/home/tronco/anaconda2/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [17]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X1[:,0],X1[:,1],X1[:,2])
plt.show()

In [ ]:
n,m = X0.shape
X_lineal = np.reshape(X0.T, (n*m))
alpha_lineal = np.reshape(alpha_sol.T, (n*m))
vector = np.hstack([X_lineal,alpha_lineal])
t = np.linspace(0,1,3)
g = lambda y,t: f(y,sigma)
matrix = odeint(g,vector,t)

In [21]:
alpha0 = np.random.random(X0.shape[0]* X0.shape[1])
alpha = shootingMethod(X0, X1, alpha0, 3, sigma)

KeyboardInterrupt: 